In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Load the data
file_path = 'Label_Data_Lily.xlsx'
df = pd.read_excel(file_path)
data = df.iloc[182001:182501]

# Function to fetch and parse website content
def fetch_website_content(url):
    try:
        response = requests.get(url, timeout=10)
        if response.history:  # Check if the URL was redirected
            final_url = response.url
        else:
            final_url = url
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup, final_url
    except requests.exceptions.RequestException:
        return None, url

# Function to check if company details match website content
def check_match(soup, business_name, address, city):
    if not soup:
        return "No"

    # Convert everything to lowercase for better matching
    business_name = business_name.lower()
    address = address.lower()
    city = city.lower()

    # Check for name, address, and city in the webpage content
    content = soup.get_text().lower()

    if (business_name in content) and (address in content or city in content):
        return "Yes"
    else:
        return "No"

# Iterate through each row and perform the check
matches = []
redirected_urls = []

for index, row in data.iterrows():
    url = row['url']  # URL is already complete with "https://"
    soup, final_url = fetch_website_content(url)

    # Check for match
    match = check_match(soup, row['Business_Name'], row['Address'], row['City'])
    matches.append(match)

    # Check if URL was redirected
    if final_url != url:
        redirected_urls.append(final_url)
    else:
        redirected_urls.append("")

# Add the results to the DataFrame
data['match'] = matches
data['redirected_url'] = redirected_urls

# Save the updated DataFrame to a new Excel file
output_path = 'Labeled_Data.xlsx'
data.to_excel(output_path, index=False)

print(f"Data has been processed and saved to {output_path}")


In [12]:
x= pd.read_excel('Labeled_Data.xlsx')
x

,index,url,Business_Name,Address,City,match,redirected_url
0,182001,https://state.nj.us,Dr Maria Campoalegre Md Pa,407 39Th St,Union City,No,https://www.nj.gov/
1,182002,https://century21gas.com,Nyce Shirt,310 Broad St Ste D,Harleysville,No,NaN
2,182003,https://hpcblawnox.org,Hoboken Presbyterian Church,921 North Ave,Pittsburgh,No,NaN
3,182004,https://carquest.com,A M. Auto Parts Ltd.,44 Main St,Souris,No,NaN
4,182005,https://laharborsportfishing.com,La Waterfront Cruises,400 Oceangate Ste 1110,Long Beach,No,https://www.laharborsportfishing.com/
...,...,...,...,...,...,...,...
494,182495,https://benjaminmoore.com,Florham Park Hardware,186 Columbia Tpke,Florham Park,No,https://www.benjaminmoore.com/en-us
495,182496,https://morgancenter.org,Adapti Cooperatives,202-105 Boulevard De La Grande-Baie Nord,La Baie,No,NaN
496,182497,https://hoovershotshots.com,Tempe Montessori School,410 S El Dorado,Mesa,No,https://www.hoovershotshots.com/
497,182498,https://lowcostdivorceny.com,Dina Degiorgio,551 Port Washington Blvd Ste 1,Port Washington,No,https://keildivorcelaw.com
